In [ ]:
!pip install -qU \
  python-dotenv \
  langchain \
  openai \
  anthropic \
  langchain-openai \
  langchain-anthropic \
  sqlalchemy \
  google-cloud-bigquery

In [ ]:
import os
from langchain_community.document_loaders import BigQueryLoader

service_account_path = './gbqkey.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=service_account_path

In [ ]:
def get_docs(query):
    loader = BigQueryLoader(query,project='rabbitpromotion')
    data = loader.load()
    return data

In [ ]:
table_1="analytics_323497507.events_20230816"
table_2="bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_20201101"

In [ ]:
query=f"""
SELECT event_name, COUNT(*) as event_count
FROM `{table_1}`
GROUP BY event_name
ORDER BY event_count DESC
LIMIT 5
"""

In [ ]:
get_docs(query)

[Document(page_content='event_name: session_start\nevent_count: 2'),
 Document(page_content='event_name: user_engagement\nevent_count: 2'),
 Document(page_content='event_name: page_view\nevent_count: 2'),
 Document(page_content='event_name: scroll\nevent_count: 2'),
 Document(page_content='event_name: first_visit\nevent_count: 1')]

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(service_account_path)
gbq_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [ ]:
gbq_client.query(query).to_dataframe()

,event_name,event_count
0,session_start,2
1,user_engagement,2
2,page_view,2
3,scroll,2
4,first_visit,1


In [ ]:
def build_schema_desc(fields, prefix=""):
    desc = []
    for f in fields:
        d = f"{prefix}- Name: {f.name}, Type: {f.field_type}, Mode: {f.mode}"
        desc.append(d)
        if f.field_type == 'RECORD':
            sub_desc = build_schema_desc(f.fields, prefix + "    ")
            desc.extend(sub_desc)
    return desc

def fetch_schemas(dataset_id, client):
    schemas = ["Detailed Schema Descriptions:"]
    simple_table_list = ["Table Identifiers:"]
    processed_prefixes = set()

    tables = client.list_tables(dataset_id)
    for table in tables:
        if table.table_id.startswith('events_') and 'events_' not in processed_prefixes:
            processed_prefixes.add('events_')
        elif table.table_id.startswith('events_intraday_') and 'events_intraday_' not in processed_prefixes:
            processed_prefixes.add('events_intraday_')
        else:
            continue

        ref = client.get_table(table)
        simple_table_list.append(f"- {ref.project}.{ref.dataset_id}.{ref.table_id}")
        schema_desc = [f"Schema for {table.table_id}:"]
        schema_desc += build_schema_desc(ref.schema)
        schema_desc.append("")

        schemas += schema_desc

        if len(processed_prefixes) == 2:
            break

    return "\n".join(simple_table_list) + "\n\n" + "\n".join(schemas)

In [ ]:
full_dataset_id_1 ="rabbitpromotion.analytics_323497507"
full_dataset_id_2="bigquery-public-data.ga4_obfuscated_sample_ecommerce"

In [ ]:
print(fetch_schemas(full_dataset_id_2,gbq_client))

Table Identifiers:
- bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_20201101

Detailed Schema Descriptions:
Schema for events_20201101:
- Name: event_date, Type: STRING, Mode: NULLABLE
- Name: event_timestamp, Type: INTEGER, Mode: NULLABLE
- Name: event_name, Type: STRING, Mode: NULLABLE
- Name: event_params, Type: RECORD, Mode: REPEATED
    - Name: key, Type: STRING, Mode: NULLABLE
    - Name: value, Type: RECORD, Mode: NULLABLE
        - Name: string_value, Type: STRING, Mode: NULLABLE
        - Name: int_value, Type: INTEGER, Mode: NULLABLE
        - Name: float_value, Type: FLOAT, Mode: NULLABLE
        - Name: double_value, Type: FLOAT, Mode: NULLABLE
- Name: event_previous_timestamp, Type: INTEGER, Mode: NULLABLE
- Name: event_value_in_usd, Type: FLOAT, Mode: NULLABLE
- Name: event_bundle_sequence_id, Type: INTEGER, Mode: NULLABLE
- Name: event_server_timestamp_offset, Type: INTEGER, Mode: NULLABLE
- Name: user_id, Type: STRING, Mode: NULLABLE
- Name: user_pseudo_id,

In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from langchain_anthropic import ChatAnthropic
llm_claude3 = ChatAnthropic(model='claude-3-opus-20240229')

from langchain_openai import ChatOpenAI
llm_gpt4 = ChatOpenAI(model="gpt-4")

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = """Based on the GA4 BigQuery schema below, write a SQL query that answers the user's question
Only return clean executable SQL without quotes:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
def get_schema(_):
    return fetch_schemas(full_dataset_id_2,gbq_client)

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm_gpt4.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [ ]:
sql_prompt="""
List the top 5 events.
"""
res=sql_response.invoke({"question": sql_prompt})
print(res)

SELECT event_name, COUNT(*) as event_count 
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_20201101` 
GROUP BY event_name 
ORDER BY event_count DESC 
LIMIT 5


In [ ]:
gbq_client.query(res).to_dataframe()

,event_name,event_count
0,page_view,9767
1,user_engagement,8667
2,view_item,3380
3,scroll,3040
4,session_start,2594


In [ ]:
final_template = """
Based on the table schema below, question, sql query, and sql response, write a natural
language response.
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}
"""
final_prompt = ChatPromptTemplate.from_template(final_template)

In [ ]:
final_chain = (
    RunnablePassthrough.assign(query=sql_response).assign(
        schema=get_schema,
        response=lambda x: get_docs(x["query"]),
    )
    | final_prompt
    | llm_gpt4
)

In [ ]:
final_chain.invoke({"question": "List the top 5 events) "})

AIMessage(content="The top 5 events in the `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_20201101` dataset are as follows: \n\n1. 'page_view' with a count of 9767 occurrences\n2. 'user_engagement' with a count of 8667 occurrences\n3. 'view_item' with a count of 3380 occurrences\n4. 'scroll' with a count of 3040 occurrences\n5. 'session_start' with a count of 2594 occurrences.", response_metadata={'token_usage': {'completion_tokens': 107, 'prompt_tokens': 1956, 'total_tokens': 2063}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None})